In [1]:
import pandas as pd
import numpy as np

In [2]:
education = pd.read_csv("../Data/WorldBank/Education_WB/edData.csv")
educationSimple = pd.read_csv("../Data/WorldBank/WB_Edu_simple/646c5220-17f9-41f4-b39f-0f8c34130df6_Data.csv")
gdpInvestment = pd.read_csv("../Data/WorldBank/GDP-Investment_WB/9570f3c5-6012-43e4-980d-8ea2ece70445_Data.csv")
childMortality = pd.read_csv("../Data/WorldBank/childMortality_WB/8a867181-d49e-48fc-b846-4dc5e7550807_Data.csv")
taxes = pd.read_csv("../Data/WorldBank/WB_taxesOfGDP/a77adba3-031c-495c-a100-610c369cdff7_Data.csv")


In [3]:
literacy = pd.read_csv("../Data/cross-country-literacy-rates.csv")

In [4]:
govExpend = pd.read_excel("../Data/IMF/gov_exp_GDP.xls")
govRevenue = pd.read_excel("../Data/IMF/govRevenuePercOfGDP.xls")
govDebt = pd.read_excel("../Data/IMF/debtPercOfGDP.xls")

In [5]:
wb_dict = dict(zip(education['Country Name'], education['Country Code']))

In [6]:
auxillaryDictionary = {
    "China, People's Republic of":'CHN',
    "Congo, Dem. Rep. of the":'COD',
    "Congo, Republic of":'COG',
    "Congo, Republic of ":'COG',
    "Côte d'Ivoire": 'CIV',
    "Egypt":'EGY',
    "Hong Kong SAR":'HKG',
    "Iran":'IRN',
    "Korea, Republic of":'KOR',
    "Lao P.D.R.":"LAO",
    "North Macedonia":"MKD",
    "North Macedonia ":"MKD",
    "Saint Vincent and the Grenadines":"VCT",
    "South Sudan, Republic of":'SSD',
    'Syria':"SYR",
    'São Tomé and Príncipe':'STP',
    'Venezuela':'VEN',
    'Yemen':'Yem',
    'Timor':'TLS',
    'Brunei'	:'BRN'
}

In [7]:
worldBankPlusDictionary = { **auxillaryDictionary, **wb_dict}

In [8]:
def imf_set_transposer(imf_data_set):
    imf_data_set.iloc[:,0]  = imf_data_set.iloc[:,0].replace(worldBankPlusDictionary)
    transposed = imf_data_set.T
    result = []
    listName = (transposed.iloc[0].name).replace("%","perc")
    listName = listName.replace(" ","_")
    for country in transposed:
        for (year, value) in enumerate(transposed.iloc[1:,country]):
            yearPublish = year+1800
            countryISO = transposed.iloc[0,country]
            if value =="no data": value = np.NaN
            result.append({
                    'year':yearPublish,
                    'country':countryISO,
                    listName:value,
            })
    return pd.DataFrame(result)


In [9]:
def drop_non_year_column(titles):
    # Get Years in dataset
    drops = []
    for title in titles:
        if not title.isdigit() : drops.append(title)
    return drops

def cut_year_title_down(title):
    # world bank columns end with [YRxxxx] EG: 1970 [YR1970]
    # This can be cut a way easily
    if title[0].isdigit() : title = title[0:4]
    return title

def transpose_world_bank_data(worldBankData):
    result = []
    
    worldBankData = pd.DataFrame(worldBankData)
    worldBankData.columns = [cut_year_title_down(x) for x in worldBankData.columns]

    years = worldBankData.columns.drop(labels=drop_non_year_column(worldBankData.columns))
    grouped = worldBankData.groupby("Series Code")
    for (name, group) in grouped.__iter__():
        transposed = group.transpose()
        Resgrouped = []
        
        for (country) in transposed[:3]:
            countryData = transposed[country]
            for year in years:
                Resgrouped.append({
                    'year':int(year),
                    'country':countryData["Country Code"],
                    name:countryData[year],
                    })

        if len(result) > 0: 
            result = pd.DataFrame(result)
            new = pd.DataFrame(Resgrouped)
            result= result.merge(new, on=["year","country"], how="outer", sort=True)
        else:
            result = pd.DataFrame(Resgrouped)
    return result

In [10]:
education_data = transpose_world_bank_data(education)

gdpInvestment_data = transpose_world_bank_data(gdpInvestment)
total = education_data.merge(gdpInvestment_data, on=["year","country"], how="outer", sort=True)

govExpend_data = imf_set_transposer(govExpend)
total = total.merge(govExpend_data, on=["year","country"], how="left", sort=True)

govRevenue_data = imf_set_transposer(govRevenue)
total = total.merge(govRevenue_data, on=["year","country"], how="left", sort=True)

govDebt_data = imf_set_transposer(govDebt)
total = total.merge(govDebt_data, on=["year","country"], how="left", sort=True)

educationSimple_data = transpose_world_bank_data(educationSimple)
total = total.merge(educationSimple_data, on=["year","country"], how="outer", sort=True)

childMortality_data = transpose_world_bank_data(childMortality)
total = total.merge(childMortality_data, on=["year","country"], how="outer", sort=True)

taxes_data = transpose_world_bank_data(taxes)
total = total.merge(taxes_data, on=["year","country"], how="outer", sort=True)


In [11]:
def fitLiteracyForEachYear(literacySet,minYear=1960, maxYear=2021):
    grouped = literacySet.groupby("Code")
    results = []
    for (name, group) in grouped.__iter__():
        year = group.iloc[:,2]
        score = group.iloc[:,3]
        m,b = np.polyfit(year, score, 1)
        for yearToCheck in range(minYear, (maxYear+1), 1):
            score = m * yearToCheck + b
            results.append({
                'year':int(yearToCheck),
                'literacy':np.clip(score,0,100),
                'country':name
            })
    return pd.DataFrame(results)


In [12]:
import warnings
warnings.filterwarnings("ignore")
literacy_data = fitLiteracyForEachYear(literacy, total["year"].min(),total["year"].max())
total = total.merge(literacy_data, on=["year","country"], how="outer", sort=True)

In [13]:
total = total.loc[~total.index.duplicated(),:].copy()

In [14]:
columnNames = {
    'NY.GDP.MKTP.CD_x':'GDP-currentDollar-outdated',
    'NY.GDP.MKTP.CD':'GDP-currentDollar',
    'NY.GDP.PCAP.CD':'GDP-PerCapita',
    'NY.GDP.PCAP.CD_x':'GDP-PerCapita-outdated',
    'NY.GDP.MKTP.KD':'GDP-In2015Dollars',
    'NY.GDP.PCAP.KD':'GDP-perCapital2015Dollar',
    'NY.GDP.PCAP.PP.KD':'GDP-perCapital2017Dollar',
    'NY.GDP.MKTP.PP.CD':'GDP-InCurrentDollars',
    'NY.GDP.MKTP.PP.KD':'GDP-In2017Dollars',
    'NY.GDP.PCAP.PP.CD':'GDP-perCapital-current',
    'NY.GDS.TOTL.ZS':'grossDomesticSavings',
    'SP.DYN.IMRT.IN':'childMortalityRatePer1000',
    'GC.TAX.TOTL.GD.ZS':'taxRevenueOfGDP',
    'NY.ADJ.ICTR.GN.ZS':'adjustedSavingsGrossSaving',
    'SE.XPD.TOTL.GD.ZS':'expenseOnEduOfGDP',
    'BN.KLT.DINV.CD':'foreigDirectInvestment',
    'BX.KLT.DINV.WD.GD.ZS':'foreigDirectInvestmentOfGDP',
    'GC.NFN.TOTL.GD.ZS':'netInvestmentOfGDP',
    
    'SE.ADT.LITR.ZS': 'adult_llit-perc',
    'UIS.XGDP.56.FSGOV':'tertEduOfGDP',
    'UIS.XGDP.23.FSGOV':'seconEduOfGDP',
    'UIS.XGDP.1.FSGOV':'primerEduOfGDP',

    "UIS.LPP.Ag15t99":	"Adul_illit_pop_prec-female",
    "UIS.LP.Ag15t99":	"Adul_illit_pop_prec",
    "UIS.LP.Ag15t99.F":	"Adul_illit_pop_prec-female",
    "UIS.LP.Ag15t99.M":	"Adul_illit_pop_prec-male",
    "UIS.LP.Ag65.F":	"Elderly_illit_pop-female",
    "UIS.LP.Ag65.M":	"Elderly_illit_pop-male",
    "UIS.ILLPOPF.AG25T64":	"Illit_pop_workForce-female-percentage",
    "UIS.ILLPOP.AG25T64":	"Illit_pop_workForce",
    "UIS.ILLPOP.AG25T64.F":	"Illit_pop_workForce-female-number",
    "UIS.ILLPOP.AG25T64.M":	"Illit_pop_workForce-male",
    "UIS.PLILLITP":	"Partic_lit_prog-illit_pop",
    "UIS.PLILLITP.F":	"Partic_lit_prog_illit_pop-female",
    "UIS.PLILLITP.M":	"Partic_lit_prog_illit_pop-male",
    "UIS.LPP.Ag15t24":	"Youth_illit_pop_prec-female",
    "UIS.LP.Ag15t24":	"Youth_illit_pop",
    "UIS.LP.Ag15t24.F":	"Youth_illit_pop_female",
    "UIS.LP.Ag15t24.M":	"Youth_illit_pop_male",
    
    "UIS.XGDP.2.FSGOV":	"Gov_expe_second_edu-of-GDP",
    "UIS.XGDP.4.FSGOV":	"Gov_expe_secodPost_edu-of-GDP",
    "UIS.XGDP.0.FSGOV":	"Gov_expe_prePrim_edu-of-GDP",
    "UIS.XGDP.1.FSGOV":	"Gov_expe_prim_edu-of-GDP",

    "UIS.XGDP.2T4.V.FSGOV": "Gov_expe_secondTer-edu-ofGDP",
    "UIS.XGDP.23.FSGOV": "Gov_expe_second_educ-perc-GDP",
    "UIS.XGDP.56.FSGOV": "Gov_expe_tert_edu-perc-GDP",

    "UIS.XGDP.3.FSGOV": "Gov_expe on upper secondary education as a percentage of GDP (%)",
    "UIS.XUNIT.GDPCAP.2.FSGOV":	"inti_gov_fund-lowSec-perc-perGDP",
    "UIS.XUNIT.GDPCAP.02.FSGOV": "inti_gov_fund-prePrim-perc-perGDP",
    "UIS.XUNIT.GDPCAP.1.FSGOV":	"inti_gov_fund-Prim-perc-perGDP",
    "UIS.XUNIT.GDPCAP.23.FSGOV": "inti_gov_fund-sec-perc-perGDP",
    "UIS.XUNIT.GDPCAP.5T8.FSGOV": "inti_gov_fund-tert-perc-GDP-perCap",
    "UIS.XUNIT.GDPCAP.3.FSGOV": "inti_gov_fund-uppSec-perc-GDP-perCap",

    "UIS.XUNIT.GDPCAP.23.FSHH": "inti_house-fund_seco_stud-per-GDP",
    "UIS.XUNIT.GDPCAP.1.FSHH": "inti_house-fund_prim_stud-per-GDP",
    "UIS.XUNIT.GDPCAP.5T8.FSHH": "inti_house-fund_tert_stud-per-GDP",

}
total = total.rename(columns=columnNames)

In [15]:
total = total.replace("..",np.nan)

In [16]:
# total.to_csv("beforeProcess.csv")

In [17]:
# total.isna().sum().sort_values(ascending=False) / 16747
# largeNan = total.isna().sum()[(total.isna().sum()/16747) > 0.9].keys()
# doppredColumns = total.drop(columns=largeNan, axis=0)

# doppredColumns.isna().sum().sort_values(ascending=False) / 16747

In [18]:
total["noGDPDummy"] = total["GDP-PerCapita"].isna()
total["noTaxDummy"] = total["taxRevenueOfGDP"].isna()

In [19]:
total = total.drop(columns=["GDP-currentDollar-outdated", "GDP-In2015Dollars","GDP-InCurrentDollars","GDP-In2017Dollars"], axis=0)

In [20]:
total = total.drop(columns=["GDP-perCapital2015Dollar","GDP-perCapital-current","GDP-perCapital2017Dollar"], axis=0)

In [21]:
dummyVariables = total[["noTaxDummy","noGDPDummy","country","year"]]
dummyVariables.to_csv("dummy_data.csv", index=False)

In [22]:
numData = total.drop(columns=["noTaxDummy","noGDPDummy"], axis=0)

In [23]:
for column in numData.columns:
    if column != "country":
        numData[column] = pd.to_numeric(numData[column])
        if column != 'year':
            numData[column+'interPolation'] = numData.groupby('country').apply(lambda group: group.interpolate())[column]

In [24]:
def drop_non_year_column(titles):
    # Get Years in dataset
    drops = []
    for title in titles:
        # if not title.endswith("interPolation") and not column.endswith('Dummy') : drops.append(title)
        if not title.endswith("interPolation") and not title.endswith('year') and not title.endswith('country') : drops.append(title)
    return drops

In [25]:
numData = numData.drop(columns=drop_non_year_column(numData.columns), axis=0)

In [26]:
dropColls = ["Illit_pop_workForceinterPolation","Illit_pop_workForce-female-numberinterPolation","Illit_pop_workForce-maleinterPolation","Illit_pop_workForce-female-percentageinterPolation","UIS.LP.AG15T24interPolation","UIS.LP.AG15T24.FinterPolation","UIS.LP.AG15T24.MinterPolation","UIS.LP.AG15T99interPolation","UIS.LP.AG15T99.FinterPolation","UIS.LP.AG15T99.MinterPolation","UIS.LP.AG65interPolation","UIS.LP.AG65.FinterPolation","UIS.LP.AG65.MinterPolation","UIS.LPP.AG15T24interPolation","UIS.LPP.AG15T99interPolation","UIS.LPP.AG65interPolation","Partic_lit_prog-illit_popinterPolation","Partic_lit_prog_illit_pop-femaleinterPolation","Partic_lit_prog_illit_pop-maleinterPolation"]

In [27]:
numData = numData.drop(columns=dropColls, axis=0)
numData = numData.dropna(thresh=5)

In [33]:
for column in numData.columns:
    if not column.endswith('Dummy') and not column.endswith('year') and not column.endswith('country') and not column.endswith('growth') and not column.endswith('PercentageGrowth'):
        numData[column+'-growth'] = numData[column].rolling(window=2).apply(lambda x: x.iloc[-1] - x.iloc[0])
        numData[column+'-3y-growth'] = numData[column].rolling(window=3).apply(lambda x: x.iloc[-1] - x.iloc[0])
        numData[column+'-5y-growth'] = numData[column].rolling(window=5).apply(lambda x: x.iloc[-1] - x.iloc[0])
        numData[column+'-PercentageGrowth'] = numData[column].rolling(window=2).apply(lambda x: ((x.iloc[-1] - x.iloc[0])/x.iloc[0]))

In [34]:
numData.to_csv("numData.csv", index=False)
total.to_csv("data.csv", index=False)